In [ ]:
# Face recognition i.e saving the face for authentication

import cv2
import numpy as np
import mediapipe as mp

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
face_mesh = mp_face.FaceMesh(static_image_mode=True,max_num_faces=1,refine_landmarks=True)


while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f = cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    h,w,_ = f.shape
    
    res = face_mesh.process(rgb)

    if res.multi_face_landmarks:
        for landmarks in res.multi_face_landmarks:
            face_vector = []

            for lm in landmarks.landmark:
                x,y = int(lm.x*w),int(lm.y*h)
                face_vector.append([x,y])
                cv2.circle(f,(x,y),1,(0,255,0),-1)

            if cv2.waitKey(1) & 0xff == ord('s'):
                np.save("authorized_face.npy",np.array(face_vector))
                print("Authoruzed Face Saved")
                break

    cv2.putText(f,"Press S to Save the face",(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,0),2)
    cv2.imshow("Face Recognition System",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break


cap.release()
cv2.destroyAllWindows()

In [ ]:
# Detecting face from saved one
import cv2
import mediapipe as mp
import numpy as np
from scipy.spatial import distance as dist

authorized_face = np.load("authorized_face.npy")

cap = cv2.VideoCapture(0)
mp_face = mp.solutions.face_mesh
mp_draw = mp.solutions.drawing_utils
face_mesh = mp_face.FaceMesh(static_image_mode=False,max_num_faces=1,refine_landmarks=True)

threshold = 50

def calculateSimilarity(face1,face2):
    if len(face1)!=len(face2):
        return float('inf')
    return np.mean([dist.euclidean(p1,p2) for p1,p2 in zip(face1,face2)])

while cap.isOpened():
    r,f = cap.read()
    if r==False:
        break
    f=cv2.flip(f,1)
    rgb = cv2.cvtColor(f,cv2.COLOR_BGR2RGB)
    h,w,_ = f.shape

    res = face_mesh.process(rgb)
    status = "Face Not Detected"

    if res.multi_face_landmarks:
        for landmarks in res.multi_face_landmarks:
            live_face = []
            for lm in landmarks.landmark:
                x,y = int(lm.x*w),int(lm.y*h)
                live_face.append([x,y])
                cv2.circle(f,(x,y),1,(0,255,0),-1)

            similarity = calculateSimilarity(authorized_face,live_face)

            if similarity<threshold:
                status = "Access Granted"
                color = (0,255,0)
            else:
                status = "Access Denied"
                color = (0,0,255)
            mp_draw.draw_landmarks(f,landmarks,mp_face.FACEMESH_CONTOURS,mp_draw.DrawingSpec(color=(0,255,0),thickness=1,circle_radius=1),mp_draw.DrawingSpec(color=(0,0,255),thickness=1))
            cv2.putText(f,f'Similarity: {similarity:.2f}',(10,60),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(255,255,0),2)
    else:
        color = (0,0,255)

    cv2.putText(f,status,(10,30),cv2.FONT_HERSHEY_COMPLEX_SMALL,1,color,1)

    cv2.imshow("Face Detection",f)
    if cv2.waitKey(25) & 0xff == ord('d'):
        break

cap.release()
cv2.destroyAllWindows()
